### Grabs the players data using the fetchplayerstats class
Enter the year you want in season and if you want Regualr Season or Playoffs in season_type

In [ ]:
from NBAData.fetchPlayersStats import FetchPlayersStats

nba = FetchPlayersStats()
data = nba.getCompleteStats(season='2024-25', season_type='Playoffs', sleep_time=1, max_workers=5)

In [42]:
from NBAData.features import *
from NBAData.playerPositions import *
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
data = pd.read_csv('CSV_FILES/REGULAR_DATA/SEASON_21.csv')

data['STARTING'] = data['START_POSITION'].apply(lambda x: 1 if x in ['G','F','C'] else 0)
data = rollingAverages(data, [2, 4, 6], stat_line='AST')
data = addLagFeatures(data, stat_line='AST')
data = HomeAwayAverages(data, stat_line='AST')
data = statAgainstTeam(data, player_id_col='PLAYER_ID', opp_col='OPP_ABBREVIATION', stat_line='AST')
data = calculateDaysOfRest(data)
data = preprocessGamesData(data)
data = add_all_opponent_features(data, stat_line='AST')
data = assign_position_with_cache(
    data, 
    cache_file='nba_positions.csv',
    max_workers=4, 
    delay_between_requests=1.5
)
data.to_csv('CSV_FILES/REGULAR_DATA/SEASON_21_ast_features.csv')
data

c:\Users\alexg\OneDrive\Documents\Prize-Picks-Prop-Predictor\NBAData\features.py:190: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_expanding_avg)
c:\Users\alexg\OneDrive\Documents\Prize-Picks-Prop-Predictor\NBAData\features.py:190: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_expanding_avg)


Loading position cache...
Loaded 992 players from cache
Found 540 unique players, 0 need to be fetched
Applying positions to dataset...
Creating position flags...
Position assignment completed!


,Unnamed: 0,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,COMMENT,OFF_RATING,E_OFF_RATING,DEF_RATING,E_DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,EFG_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,POSS,PACE_PER40,E_USG_PCT,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE,STARTING,MIN_ROLL_AVG_2,AST_ROLL_AVG_2,FGA_ROLL_AVG_2,FGM_ROLL_AVG_2,FG_PCT_ROLL_AVG_2,FG3A_ROLL_AVG_2,FG3M_ROLL_AVG_2,FG3_PCT_ROLL_AVG_2,FTM_ROLL_AVG_2,FTA_ROLL_AVG_2,FT_PCT_ROLL_AVG_2,USG_PCT_ROLL_AVG_2,AST_PCT_ROLL_AVG_2,AST_TOV_ROLL_AVG_2,TS_PCT_ROLL_AVG_2,EFG_PCT_ROLL_AVG_2,PIE_ROLL_AVG_2,PLUS_MINUS_ROLL_AVG_2,TEAM_FG_PCT_ROLL_AVG_2,TEAM_FGM_ROLL_AVG_2,TEAM_AST_ROLL_AVG_2,TEAM_TOV_ROLL_AVG_2,TEAM_PACE_ROLL_AVG_2,TEAM_PTS_ROLL_AVG_2,OPP_DEF_RATING_ROLL_AVG_2,OPP_STL_ROLL_AVG_2,OPP_PACE_ROLL_AVG_2,MIN_ROLL_AVG_4,AST_ROLL_AVG_4,FGA_ROLL_AVG_4,FGM_ROLL_AVG_4,FG_PCT_ROLL_AVG_4,FG3A_ROLL_AVG_4,FG3M_ROLL_AVG_4,FG3_PCT_ROLL_AVG_4,FTM_ROLL_AVG_4,FTA_ROLL_AVG_4,FT_PCT_ROLL_AVG_4,USG_PCT_ROLL_AVG_4,AST_PCT_ROLL_AVG_4,AST_TOV_ROLL_AVG_4,TS_PCT_ROLL_AVG_4,EFG_PCT_ROLL_AVG_4,PIE_ROLL_AVG_4,PLUS_MINUS_ROLL_AVG_4,TEAM_FG_PCT_ROLL_AVG_4,TEAM_FGM_ROLL_AVG_4,TEAM_AST_ROLL_AVG_4,TEAM_TOV_ROLL_AVG_4,TEAM_PACE_ROLL_AVG_4,TEAM_PTS_ROLL_AVG_4,OPP_DEF_RATING_ROLL_AVG_4,OPP_STL_ROLL_AVG_4,OPP_PACE_ROLL_AVG_4,MIN_ROLL_AVG_6,AST_ROLL_AVG_6,FGA_ROLL_AVG_6,FGM_ROLL_AVG_6,FG_PCT_ROLL_AVG_6,FG3A_ROLL_AVG_6,FG3M_ROLL_AVG_6,FG3_PCT_ROLL_AVG_6,FTM_ROLL_AVG_6,FTA_ROLL_AVG_6,FT_PCT_ROLL_AVG_6,USG_PCT_ROLL_AVG_6,AST_PCT_ROLL_AVG_6,AST_TOV_ROLL_AVG_6,TS_PCT_ROLL_AVG_6,EFG_PCT_ROLL_AVG_6,PIE_ROLL_AVG_6,PLUS_MINUS_ROLL_AVG_6,TEAM_FG_PCT_ROLL_AVG_6,TEAM_FGM_ROLL_AVG_6,TEAM_AST_ROLL_AVG_6,TEAM_TOV_ROLL_AVG_6,TEAM_PACE_ROLL_AVG_6,TEAM_PTS_ROLL_AVG_6,OPP_DEF_RATING_ROLL_AVG_6,OPP_STL_ROLL_AVG_6,OPP_PACE_ROLL_AVG_6,AST_LAG_1,AST_LAG_2,AST_LAG_3,AST_LAG_4,PLAYER_HOME_AVG_AST,PLAYER_AWAY_AVG_AST,MATCHUP_AVG_AST_LAST_3,GAMES_VS_OPP,DAYS_OF_REST,DAYS_AGO,OPP_ROLL_OPP_DEF_RATING_5,OPP_ROLL_OPP_STL_5,OPP_ROLL_OPP_BLK_5,NORMALIZED_DEF_RATING,OPP_DEF_STRENGTH,DEF_CATEGORY,AST_VS_DEF,AST_PCT_VS_DEF,AST_TOV_VS_DEF,USG_PCT_VS_DEF,PACE_VS_DEF,POSS_VS_DEF,OFF_RATING_VS_DEF,GUARD,FORWARD,CENTER
6,6,LeBron James,2544,LAL vs. LAC,LAL,1610612747,LAC,1,22000002,2020-12-22,L,28,22,5,5,7,17,0.412,3,8,0.375,5,7,0.714,2,3,0,0,4,2,2,31.5,1.096,0.500000,F,NaN,103.1,103.8,100.0,103.3,3.1,0.067,0.094,0.081,0.294,0.500,1.25,0.343,0.548,106.29,108.36,0.122,64,90.30,0.345,"DEC 22, 2020",240,38,81,0.469,9,29,0.310,24,31,0.774,8,37,45,22,4,2,19,20,109,103.180613,10.0,3.0,40.0,0.473118,1610612746,103.180613,105.64,106.00,106.36,1,28.0,5.0,17.0,7.0,0.41,8.0,3.0,0.38,5.0,7.0,0.71,0.34,0.29,1.25,0.55,0.50,0.12,2.0,0.47,38.0,22.0,19.0,105.64,109.0,103.18,10.0,106.36,28.00,5.00,17.00,7.00,0.41,8.00,3.00,0.38,5.0,7.00,0.71,0.34,0.29,1.25,0.55,0.50,0.12,2.00,0.47,38.00,22.00,19.00,105.64,109.00,103.18,10.00,106.36,28.00,5.00,17.00,7.00,0.41,8.00,3.00,0.38,5.00,7.00,0.71,0.34,0.29,1.25,0.55,0.50,0.12,2.00,0.47,38.00,22.00,19.00,105.64,109.00,103.18,10.00,106.36,NaN,NaN,NaN,NaN,5.00,8.00,5.00,1,NaN,1647,103.18,10.0,3.0,0.692062,0.876691,1,-0.322,-0.283,-0.454,0.072,0.082,-0.071,-0.064,0,1,0
18,18,Jarrett Allen,1628386,BKN vs. GSW,BKN,1610612751,GSW,1,22000001,2020-12-22,W,20,8,2,7,2,3,0.667,0,0,NaN,4,5,0.800,3,4,1,2,2,0,12,26.4,1.538,0.666667,NaN,NaN,113.0,117.2,88.9,83.0,24.2,0.150,0.148,0.149,0.154,0.667,1.00,0.143,0.769,113.24,111.33,0.188,46,92.78,0.148,"DEC 22, 2020",240,42,92,0.457,15,35,0.429,26,32,0.813,13,44,57,24,11,7,20,22,125,110.541210,6.0,6.0,47.0,0.373737,1610612744

In [43]:
from NBAData.features import *
from NBAData.playerPositions import *
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
data = pd.read_csv('CSV_FILES/PLAYOFF_DATA/PLAYOFFS_21.csv')

data['STARTING'] = data['START_POSITION'].apply(lambda x: 1 if x in ['G','F','C'] else 0)
data = rollingAverages(data, [2, 4, 6], stat_line='AST')
data = addLagFeatures(data, stat_line='AST')
data = HomeAwayAverages(data, stat_line='AST')
data = statAgainstTeam(data, player_id_col='PLAYER_ID', opp_col='OPP_ABBREVIATION', stat_line='AST')
data = calculateDaysOfRest(data)
data = preprocessGamesData(data)
data = add_all_opponent_features(data, stat_line='AST')
data = assign_position_with_cache(
    data, 
    cache_file='nba_positions.csv',
    max_workers=4, 
    delay_between_requests=1.5
)
data.to_csv('CSV_FILES/PLAYOFF_DATA/PLAYOFFS_21_ast_features.csv')
data

c:\Users\alexg\OneDrive\Documents\Prize-Picks-Prop-Predictor\NBAData\features.py:190: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_expanding_avg)
c:\Users\alexg\OneDrive\Documents\Prize-Picks-Prop-Predictor\NBAData\features.py:190: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_expanding_avg)


Loading position cache...
Loaded 992 players from cache
Found 239 unique players, 0 need to be fetched
Applying positions to dataset...
Creating position flags...
Position assignment completed!


,Unnamed: 0,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,COMMENT,OFF_RATING,E_OFF_RATING,DEF_RATING,E_DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,EFG_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,POSS,PACE_PER40,E_USG_PCT,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE,STARTING,MIN_ROLL_AVG_2,AST_ROLL_AVG_2,FGA_ROLL_AVG_2,FGM_ROLL_AVG_2,FG_PCT_ROLL_AVG_2,FG3A_ROLL_AVG_2,FG3M_ROLL_AVG_2,FG3_PCT_ROLL_AVG_2,FTM_ROLL_AVG_2,FTA_ROLL_AVG_2,FT_PCT_ROLL_AVG_2,USG_PCT_ROLL_AVG_2,AST_PCT_ROLL_AVG_2,AST_TOV_ROLL_AVG_2,TS_PCT_ROLL_AVG_2,EFG_PCT_ROLL_AVG_2,PIE_ROLL_AVG_2,PLUS_MINUS_ROLL_AVG_2,TEAM_FG_PCT_ROLL_AVG_2,TEAM_FGM_ROLL_AVG_2,TEAM_AST_ROLL_AVG_2,TEAM_TOV_ROLL_AVG_2,TEAM_PACE_ROLL_AVG_2,TEAM_PTS_ROLL_AVG_2,OPP_DEF_RATING_ROLL_AVG_2,OPP_STL_ROLL_AVG_2,OPP_PACE_ROLL_AVG_2,MIN_ROLL_AVG_4,AST_ROLL_AVG_4,FGA_ROLL_AVG_4,FGM_ROLL_AVG_4,FG_PCT_ROLL_AVG_4,FG3A_ROLL_AVG_4,FG3M_ROLL_AVG_4,FG3_PCT_ROLL_AVG_4,FTM_ROLL_AVG_4,FTA_ROLL_AVG_4,FT_PCT_ROLL_AVG_4,USG_PCT_ROLL_AVG_4,AST_PCT_ROLL_AVG_4,AST_TOV_ROLL_AVG_4,TS_PCT_ROLL_AVG_4,EFG_PCT_ROLL_AVG_4,PIE_ROLL_AVG_4,PLUS_MINUS_ROLL_AVG_4,TEAM_FG_PCT_ROLL_AVG_4,TEAM_FGM_ROLL_AVG_4,TEAM_AST_ROLL_AVG_4,TEAM_TOV_ROLL_AVG_4,TEAM_PACE_ROLL_AVG_4,TEAM_PTS_ROLL_AVG_4,OPP_DEF_RATING_ROLL_AVG_4,OPP_STL_ROLL_AVG_4,OPP_PACE_ROLL_AVG_4,MIN_ROLL_AVG_6,AST_ROLL_AVG_6,FGA_ROLL_AVG_6,FGM_ROLL_AVG_6,FG_PCT_ROLL_AVG_6,FG3A_ROLL_AVG_6,FG3M_ROLL_AVG_6,FG3_PCT_ROLL_AVG_6,FTM_ROLL_AVG_6,FTA_ROLL_AVG_6,FT_PCT_ROLL_AVG_6,USG_PCT_ROLL_AVG_6,AST_PCT_ROLL_AVG_6,AST_TOV_ROLL_AVG_6,TS_PCT_ROLL_AVG_6,EFG_PCT_ROLL_AVG_6,PIE_ROLL_AVG_6,PLUS_MINUS_ROLL_AVG_6,TEAM_FG_PCT_ROLL_AVG_6,TEAM_FGM_ROLL_AVG_6,TEAM_AST_ROLL_AVG_6,TEAM_TOV_ROLL_AVG_6,TEAM_PACE_ROLL_AVG_6,TEAM_PTS_ROLL_AVG_6,OPP_DEF_RATING_ROLL_AVG_6,OPP_STL_ROLL_AVG_6,OPP_PACE_ROLL_AVG_6,AST_LAG_1,AST_LAG_2,AST_LAG_3,AST_LAG_4,PLAYER_HOME_AVG_AST,PLAYER_AWAY_AVG_AST,MATCHUP_AVG_AST_LAST_3,GAMES_VS_OPP,DAYS_OF_REST,DAYS_AGO,OPP_ROLL_OPP_DEF_RATING_5,OPP_ROLL_OPP_STL_5,OPP_ROLL_OPP_BLK_5,NORMALIZED_DEF_RATING,OPP_DEF_STRENGTH,DEF_CATEGORY,AST_VS_DEF,AST_PCT_VS_DEF,AST_TOV_VS_DEF,USG_PCT_VS_DEF,PACE_VS_DEF,POSS_VS_DEF,OFF_RATING_VS_DEF,GUARD,FORWARD,CENTER
18,18,James Harden,201935,BKN vs. BOS,BKN,1610612751,BOS,1,42000111,2021-05-22,W,36,21,8,9,5,13,0.385,2,8,0.25,9,10,0.9,0,9,4,1,3,2,2,55.8,1.207,0.461538,G,NaN,104.2,106.7,104.3,98.9,-0.1,0.000,0.220,0.120,0.381,0.462,2.67,0.263,0.603,94.14,92.71,0.235,71,77.26,0.275,"MAY 22, 2021",240,35,84,0.417,8,34,0.235,26,29,0.897,14,36,50,18,7,7,11,22,104,110.921502,6.0,12.0,40.0,0.369048,1.610613e+09,110.921502,93.76,95.10,96.44,1,36.0,8.0,13.0,5.0,0.38,8.0,2.0,0.25,9.0,10.0,0.90,0.26,0.38,2.67,0.60,0.46,0.24,2.0,0.42,35.0,18.0,11.0,93.76,104.0,110.92,6.0,96.44,36.00,8.00,13.00,5.00,0.38,8.00,2.00,0.25,9.00,10.00,0.90,0.26,0.38,2.67,0.60,0.46,0.24,2.00,0.42,35.00,18.00,11.00,93.76,104.0,110.92,6.0,96.44,36.00,8.00,13.00,5.0,0.38,8.00,2.00,0.25,9.00,10.00,0.90,0.26,0.38,2.67,0.60,0.46,0.24,2.00,0.42,35.00,18.00,11.00,93.76,104.00,110.92,6.00,96.44,NaN,NaN,NaN,NaN,8.00,10.00,8.00,1,NaN,1496,110.92,6.0,12.0,0.109626,1.239541,1,0.032,0.191,0.051,0.143,0.002,-0.047,-0.102,1,0,0
54,54,JaMychal Green,203210,DEN vs. POR,DEN,1610612743,POR,1,42000161,2021-05-22,L,18,0,3,1,0,1,0.000,0,1,0.00,0,0,NaN,0,1,0,1,0,2,-6,8.7,0.000,0.000000,NaN,NaN,127.3,127.3,145.5,142.7,-18.2,0.000,0.071,0.038,0.158,0.000,0.00,0.028,0.000,90.79,89.91,0.018,33,74.93,0.029,"MAY 22, 2021",240,47,93,0.505,11,36,0.306,4,8,0.500,11,37,48,22,3,7,8,18,109,116.552609,3.0,2.0,39.0,0.472527,1.610613e+09,116.552609,9

In [89]:
pd.set_option('display.max_columns', None)

s21_playoffs = pd.read_csv('CSV_FILES/PLAYOFF_DATA/PLAYOFFS_21_ast_FEATURES.csv')
s21_playoffs = assign_playoff_series_info(s21_playoffs)
s21_playoffs['IS_PLAYOFF'] = 1
s21_regular = pd.read_csv('CSV_FILES/REGULAR_DATA/SEASON_21_ast_FEATURES.csv')
s21_regular['IS_PLAYOFF'] = 0
s21 = pd.concat([s21_playoffs, s21_regular])
s21['GAME_DATE'] = pd.to_datetime(s21['GAME_DATE'])

s22_playoffs = pd.read_csv('CSV_FILES/PLAYOFF_DATA/PLAYOFFS_22_ast_FEATURES.csv')
s22_playoffs = assign_playoff_series_info(s22_playoffs)
s22_playoffs['IS_PLAYOFF'] = 1
s22_regular = pd.read_csv('CSV_FILES/REGULAR_DATA/SEASON_22_ast_FEATURES.csv')
s22_regular['IS_PLAYOFF'] = 0
s22 = pd.concat([s22_playoffs, s22_regular])
s22['GAME_DATE'] = pd.to_datetime(s22['GAME_DATE'])

s23_playoffs = pd.read_csv('CSV_FILES/PLAYOFF_DATA/PLAYOFFS_23_ast_FEATURES.csv')  
s23_playoffs = assign_playoff_series_info(s23_playoffs)
s23_playoffs['IS_PLAYOFF'] = 1
s23_regular = pd.read_csv('CSV_FILES/REGULAR_DATA/SEASON_23_ast_FEATURES.csv')
s23_regular['IS_PLAYOFF'] = 0
s23 = pd.concat([s23_playoffs, s23_regular])
s23['GAME_DATE'] = pd.to_datetime(s23['GAME_DATE'])

s24_playoffs = pd.read_csv('CSV_FILES/PLAYOFF_DATA/PLAYOFFS_24_ast_FEATURES.csv')
s24_playoffs = assign_playoff_series_info(s24_playoffs)
s24_playoffs['IS_PLAYOFF'] = 1
s24_regular = pd.read_csv('CSV_FILES/REGULAR_DATA/SEASON_24_ast_FEATURES.csv')
s24_regular['IS_PLAYOFF'] = 0
s24 = pd.concat([s24_playoffs, s24_regular])
s24['GAME_DATE'] = pd.to_datetime(s24['GAME_DATE'])

df = pd.concat([s21,s22,s23,s24])
df = df.sort_values(by='GAME_DATE', ascending=False)
df

In [91]:
features = [
    'MIN','FGA', 'FTA', 'FG3A','FG_PCT', 'FT_PCT', 'FG3_PCT', 'REB','OREB', 'DREB', 'STL', 'BLK', 'TOV', 'PF',
    'OFF_RATING','E_OFF_RATING', 'DEF_RATING', 'E_DEF_RATING', 'NET_RATING', 'PointsPerShot', 'EFG_PCT',
    'AST_PCT', 'AST_TOV','USG_PCT', 'TS_PCT','PACE', 'PIE', 'POSS', 'E_USG_PCT', 'PLUS_MINUS',
    'TEAM_FGA', 'TEAM_FG3A','TEAM_FG_PCT','TEAM_FG3_PCT','TEAM_AST', 'TEAM_REB', 'TEAM_STL', 'TEAM_BLK', 
    'TEAM_OFF_RATING', 'TEAM_PACE', 'TEAM_PTS',
    #grab oppenents avgerages for the game
    'OPP_PACE', 'OPP_DEF_RATING','OPP_STL', 'OPP_BLK', 'OPP_REB', 'OPP_FG_PCT',
    #rolling averages
    'MIN_ROLL_AVG_2', 'AST_ROLL_AVG_2', 'FGA_ROLL_AVG_2', 'FGM_ROLL_AVG_2',
    'FG_PCT_ROLL_AVG_2', 'FG3A_ROLL_AVG_2', 'FG3M_ROLL_AVG_2', 'FG3_PCT_ROLL_AVG_2',
    'FTM_ROLL_AVG_2', 'FTA_ROLL_AVG_2', 'FT_PCT_ROLL_AVG_2', 'USG_PCT_ROLL_AVG_2',
    'AST_PCT_ROLL_AVG_2', 'AST_TOV_ROLL_AVG_2', 'TS_PCT_ROLL_AVG_2',
    'EFG_PCT_ROLL_AVG_2', 'PIE_ROLL_AVG_2', 'PLUS_MINUS_ROLL_AVG_2',
    'TEAM_FG_PCT_ROLL_AVG_2', 'TEAM_FGM_ROLL_AVG_2', 'TEAM_AST_ROLL_AVG_2',
    'TEAM_TOV_ROLL_AVG_2', 'TEAM_PACE_ROLL_AVG_2', 'TEAM_PTS_ROLL_AVG_2',
    'OPP_DEF_RATING_ROLL_AVG_2', 'OPP_STL_ROLL_AVG_2', 'OPP_PACE_ROLL_AVG_2',
    'MIN_ROLL_AVG_4', 'AST_ROLL_AVG_4', 'FGA_ROLL_AVG_4', 'FGM_ROLL_AVG_4',
    'FG_PCT_ROLL_AVG_4', 'FG3A_ROLL_AVG_4', 'FG3M_ROLL_AVG_4', 'FG3_PCT_ROLL_AVG_4',
    'FTM_ROLL_AVG_4', 'FTA_ROLL_AVG_4', 'FT_PCT_ROLL_AVG_4', 'USG_PCT_ROLL_AVG_4',
    'AST_PCT_ROLL_AVG_4', 'AST_TOV_ROLL_AVG_4', 'TS_PCT_ROLL_AVG_4',
    'EFG_PCT_ROLL_AVG_4', 'PIE_ROLL_AVG_4', 'PLUS_MINUS_ROLL_AVG_4',
    'TEAM_FG_PCT_ROLL_AVG_4', 'TEAM_FGM_ROLL_AVG_4', 'TEAM_AST_ROLL_AVG_4',
    'TEAM_TOV_ROLL_AVG_4', 'TEAM_PACE_ROLL_AVG_4', 'TEAM_PTS_ROLL_AVG_4',
    'OPP_DEF_RATING_ROLL_AVG_4', 'OPP_STL_ROLL_AVG_4', 'OPP_PACE_ROLL_AVG_4',
    'MIN_ROLL_AVG_6', 'AST_ROLL_AVG_6', 'FGA_ROLL_AVG_6', 'FGM_ROLL_AVG_6',
    'FG_PCT_ROLL_AVG_6', 'FG3A_ROLL_AVG_6', 'FG3M_ROLL_AVG_6', 'FG3_PCT_ROLL_AVG_6',
    'FTM_ROLL_AVG_6', 'FTA_ROLL_AVG_6', 'FT_PCT_ROLL_AVG_6', 'USG_PCT_ROLL_AVG_6',
    'AST_PCT_ROLL_AVG_6', 'AST_TOV_ROLL_AVG_6', 'TS_PCT_ROLL_AVG_6',
    'EFG_PCT_ROLL_AVG_6', 'PIE_ROLL_AVG_6', 'PLUS_MINUS_ROLL_AVG_6',
    'TEAM_FG_PCT_ROLL_AVG_6', 'TEAM_FGM_ROLL_AVG_6', 'TEAM_AST_ROLL_AVG_6',
    'TEAM_TOV_ROLL_AVG_6', 'TEAM_PACE_ROLL_AVG_6', 'TEAM_PTS_ROLL_AVG_6',
    'OPP_DEF_RATING_ROLL_AVG_6', 'OPP_STL_ROLL_AVG_6', 'OPP_PACE_ROLL_AVG_6',
    'AST_LAG_1', 'AST_LAG_2', 'AST_LAG_3', 'AST_LAG_4',
    'PLAYER_HOME_AVG_AST', 'PLAYER_AWAY_AVG_AST', 'MATCHUP_AVG_AST_LAST_3',
    #ill input these during prediction
    'GUARD', 'FORWARD', 'CENTER', 'STARTING', 'DAYS_OF_REST', 'HOME_GAME', 'IS_PLAYOFF', 'Series', 'GameInSeries'
]

In [92]:
from Models.xgboost_model import train_xgb_model
X = df[features].astype(float)  # Convert all features to float
y = df['AST'].astype(float)

model = train_xgb_model(X, y, stat_line='AST')
model


Fitting 3 folds for each of 50 candidates, totalling 150 fits

Model Performance Metrics for AST:
R2 Score: 0.9905
MAE: 0.1439
RMSE: 0.2507

Best Parameters: {'subsample': 0.8, 'reg_lambda': 10, 'reg_alpha': 1, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 5, 'learning_rate': 0.05, 'gamma': 0.1, 'colsample_bytree': 0.7}
Model saved to AST_xgb_model.pkl


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.7
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None
